# Simularium Conversion Tutorial : CytoSim Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.cytosim import CytosimConverter, CytosimData, CytosimObjectInfo
from simulariumio import MetaData, DisplayData, DISPLAY_TYPE, ModelMetaData, InputFileData

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `CytosimConverter` consumes spatiotemporal data from CytoSim. 

We're working to improve performance for the Cytosim converter, and also working with the Cytosim authors to add the ability to output Simularium files directly from Cytosim. For now, if you find the conversion process is too slow, you can try to record less data from Cytosim, for example record at a larger timestep by adjusting `nb_frames` in the `run` block of your Cytosim `config.cym` file (https://gitlab.com/f.nedelec/cytosim/-/blob/8feaf45297c3f5180d24889909e3a5251a7adb1a/doc/tutorials/tuto_introduction.md).

To see how to generate the Cytosim output .txt files you need, check Cytosim documentation here: https://gitlab.com/f.nedelec/cytosim/-/blob/8feaf45297c3f5180d24889909e3a5251a7adb1a/doc/sim/report.md
* for Fibers, use the command `./report fiber:points > fiber_points.txt`, which will create `fiber_points.txt`
* for Solids, use the command `./report solid > solids.txt`, which will create `solids.txt`
* for Singles, use the command `./report single:position > singles.txt`, which will create `singles.txt`
* for Couples, use the command `./report couple:state > couples.txt`, which will create `couples.txt`
    * in some versions of Cytosim, state is not a reporting option. In this case you can use `./report couple:[name of your couple] > couples_[name of your couple].txt` and provide a filepath for each type of couple in your data. If this is necessary, you should also check the position XYZ columns in your `couples.txt` file and override **position_indices** if they aren't at \[2, 3, 4\]

The converter requires a `CytosimData` object as parameter ([see documentation](https://simularium.github.io/simulariumio/simulariumio.cytosim.html#simulariumio.cytosim.cytosim_data.CytosimData)).

If you'd like to specify PDB or OBJ files or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://simularium.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


In [2]:
box_size = 2.

example_data = CytosimData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        scale_factor=10.0,
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/simularium/simulariumio",
            source_code_license_url="https://github.com/simularium/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    object_info={
        "fibers" : CytosimObjectInfo(
            cytosim_file=InputFileData(
                file_path="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/fiber_points.txt",
            ),
            display_data={
                1 : DisplayData(
                    name="microtubule",
                    radius=0.01,
                    display_type=DISPLAY_TYPE.FIBER
                ),
                2 : DisplayData(
                    name="actin",
                    radius=0.01,
                    display_type=DISPLAY_TYPE.FIBER
                )
            }
        ),
        "solids" : CytosimObjectInfo(
            cytosim_file=InputFileData(
                file_path="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/solids.txt",
            ),
            display_data={
                1 : DisplayData(
                    name="aster",
                    radius=0.1,
                    display_type=DISPLAY_TYPE.SPHERE
                ),
                2 : DisplayData(
                    name="vesicle",
                    radius=0.02,
                    display_type=DISPLAY_TYPE.SPHERE
                )
            }
        ),
        "singles" : CytosimObjectInfo(
            cytosim_file=InputFileData(
                file_path="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/singles.txt",
            ),
            display_data={
                1 : DisplayData(
                    name="dynein",
                    radius=0.01,
                    display_type=DISPLAY_TYPE.PDB,
                    url="https://files.rcsb.org/download/3VKH.pdb",
                    color="#f4ac1a",
                ),
                2 : DisplayData(
                    name="kinesin",
                    radius=0.01,
                    display_type=DISPLAY_TYPE.PDB,
                    url="https://files.rcsb.org/download/3KIN.pdb",
                    color="#0080ff",
                )
            }
        ),
        "couples" : CytosimObjectInfo(
            cytosim_file=InputFileData(
                file_path="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/couples.txt",
            ),
            display_data={
                1 : DisplayData(
                    name="motor complex",
                    radius=0.01,
                    color="#bf95d4",
                    display_type=DISPLAY_TYPE.SPHERE
                )
            },
            position_indices=[3, 4, 5]
        )
    },
)

## Convert and save as .simularium file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
CytosimConverter(example_data).save("example_cytosim")

Reading Cytosim Data -------------
Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to example_cytosim.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.